In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
path = '/home/jubi/Documents/Pesquisa/Processo seletivo /Eleflow/'
sys.path.append(path)
import netflix_functions as nf

In [2]:
df = pd.read_excel(path+'dataset_netflix.xlsx',engine='openpyxl')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jubi/Documents/Pesquisa/Processo seletivo /Eleflow/dataset_netflix.xlsx'

## Tratamento de dados (Pre-processing)

### Remoção de dados

Os filmes sem avaliação ou sem data de adição no catálogo(12), com avaliação igual a zero (330) e com data de adição no catálogo anterior a data do lançamento (8) foram removidos. Esses dados foram removidos pela não usabilidade nas etapas de treino e teste (filmes sem avaliação) ou por apresentar dados inconsistentes.

In [ ]:
df = df[(df['rating']!=0) & (df['rating'].isnull() == False)] 
df.dropna(inplace=True,subset=['date_added'])

In [ ]:
df['year'] = df['date_added'].dt.year
df['years_since_release'] = df['year']-df['release_year']
df = df[df['years_since_release']>=0]

### Encoding

Filmes foram transformados em 1 e séries em 0.

Duration foi transformado em int, e as linhas que possuiam season transformadas em np.nan.

Mantive linhas sem informações de cast como 'bam' e sem informação de country como 'None', para diretores transformei os que estavam faltando dados para 'no director'.

In [ ]:
df['type'] = df['type'].apply(nf.movie_tv)
df['type'].head()

In [ ]:
df['duration'] = df['duration'].apply(nf.time)
df['duration'].head()

In [ ]:
df['director'] = df.director.fillna('no director')
df['director'].head()

## Feature Engeneering

### Pré divisão em treino e teste

Adição do ano de lançamento na plataforma 'year' e quantos anos demorou para ser lançado na plataforma desde o lançamento do filme 'years_since_release'.

In [ ]:
df['year'] = df['date_added'].dt.year
df['years_since_release'] = df['year']-df['release_year']

### Durante treino e test

In [ ]:
df = pd.read_excel(path+'dataset_netflix.xlsx',engine='openpyxl')
df = nf.preprocessing(df,enrich=False)

variables = ['power listed_in','power director', 'power cast', 'power country','size cast',
             'size listed_in','size director','type','release_year','duration','years_since_release','year']

tcv = TimeSeriesSplit(n_splits=4)

X, y = df, df['rating']

for train_index, test_index in tcv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    X_train,y_train,X_test,y_test = nf.feature_eng_enc(X_train,y_train,X_test,y_test,3,percentile=25)    
    X_train = X_train[variables]
    X_test = X_test[variables]

Variáveis com size é o número de atóres, diretores e países de cada linha.

Variáveis com power são obtidas a partir da média do desempenho e cada um dos atóres/diretores/países, é colocado uma limitação dada por uma variável freq, no número mínimo de aparições de um ator por exemplo para ele possuir um valor na power.

Rating é substituido por 0 ou 1 dependendo do segundo quartil (percentil 50) dos dados de treino.
A escolha do quartil depende fortemente da estratégia de negócios da empresa.

In [ ]:
variables = ['power listed_in','power director', 'power cast', 'power country','size cast',
             'size listed_in','size director','type','release_year','duration','years_since_release','year']

display(X_train[variables].head())
display(y_train.head())

## Divisão de treino e teste

Para a divisão do dataset primeiro assumi que os dados utilizando para o treino precisam ser anteriores aos dados de teste, logo date_added no dataset de treino < date_added no dataset de teste, sendo assim utilizei o TimeSeriesSplit para dividir os dados corretamente.

In [ ]:

fig, axes = plt.subplots(4, 1, figsize=(15, 20))  
for i, (train_index, test_index) in enumerate(tcv.split(df)):  
    sns.lineplot(data=df, x='date_added', y=[1]*len(df), ax=axes[i], label='no_use',color="0.8")  
    sns.lineplot(data=df.iloc[train_index], x='date_added', y=[1]*len(df.iloc[train_index]), ax=axes[i], label='train',color="b")  
    sns.lineplot(data=df.iloc[test_index], x='date_added', y=[1]*len(df.iloc[test_index]), ax=axes[i], label='test',color="r")  

plt.legend()  
plt.show() 

## Resultado do modelo

In [ ]:
import netflix_model as nm
f1_test, f1_train, precision_test, precision_train, recall_test, recall_train, last_conf = nm.model_results()

In [ ]:
test_order = [1,2,3,4]
test_data = pd.DataFrame({
    'Amostra de teste': test_order, 
    'F1': f1_test,
    'Precisão': precision_test,
    'Recall': recall_test})


train_data = pd.DataFrame({
    'Amostra de treino': test_order, 
    'F1': f1_train,
    'Precisão': precision_train,
    'Recall': recall_train})
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(10, 10))  
sns.lineplot(x='Amostra de teste', y='value',ax=axes[1], hue='variable', 
             data=pd.melt(test_data, ['Amostra de teste']))
plt.title('Avaliação modelo (Teste)')

sns.lineplot(x='Amostra de treino', y='value', ax=axes[0], hue='variable', 
             data=pd.melt(train_data, ['Amostra de treino'])).set_title('Avaliação modelo (Treino)',ax=axes[0])
plt.show()

### Confusion Matrix da última amostra de teste

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
fig, axes = plt.subplots(1, 1, figsize=(10, 10))  
ConfusionMatrixDisplay(last_conf,display_labels=['Filme deve ser adicionado',
                                                'Filme não deve ser adicionado']).plot(ax = axes,colorbar=False)
plt.show()

## O que fazer para melhorar o modelo

### Melhorias internas:

1- Identificação do país de quem está avaliando o filme. 

2- Número de pessoas que avaliaram o filme.

3- Probabilidade desse filme ser indicado para alguém.

4- Número de pessoas que assistiram.

5- Objetivo de negócios. Ex: Se é desejado fazer um aumento, redução ou manutenção do catálogo.

6- Uso de graphos para relações entre atores/diretores que trabalharam juntos.

7- Avaliação individual de cada usuário, com informações do usuário.

### Melhorias por enriquecimento

1- Quantidade faturada pelo filme.

2- Custos com divulgação.

3- Produtora do filme. 

4- Tempo de produção do filme.

5- Data do lançamento

6- Avaliação em sites como rotten tomatoes ou IMDB.

7- Geração de novas variáveis "power" utilizando os ratings do IMDB/Rotten Tomatoes

8- Filme direto para home video ou não?